# Задача 2 任务 2


1.Сгенерируйте, содержащий 50000 32-битных случайных целых чисел, каждое число на отдельной строке.

In [ ]:
import random

with open('data2.txt', 'w') as file:
    for _ in range(50000):
        num = random.randint(-2147483648, 2147483647)
        file.write(f"{num}\n")


2.Посчитать, какое суммарное количество простых множителей присутствует при факторизации всех чисел. Например, пусть всего два числа: 6 и 8. 6 = 2 * 3, 8 = 2 * 2 * 2. Ответ 5.простым последовательным алгоритмом


In [6]:
def calculate(num):
    factors = []
    i = 2
    while i * i <= num:
        if num % i == 0:
            factors.append(i)
            num = num // i
        else:
            i += 1
    if num > 1:
        factors.append(num)
    return len(factors)

# 打开包含随机整数的文件
with open('data2.txt', 'r') as file:
    result = 0
    for line in file:
        number = int(line.strip())
        result += calculate(number)

print("总质因数个数 Количество факторов общего качества：", result)


总质因数个数 Количество факторов общего качества： 102898


3.многопоточно (на CPython - multiprocessing), с использованием примитивов синхронизации


# Поскольку я не мог запустить multiprocessing в Jupyter, я поместил этот блок кода в файл task22.py.

4.с помощью Dask

In [9]:
import dask.dataframe as dd

def calculate(num):
    factors = []
    i = 2
    while i * i <= num:
        if num % i == 0:
            factors.append(i)
            num = num // i
        else:
            i += 1
    if num > 1:
        factors.append(num)
    return len(factors)  
#  读取文件
df = dd.read_csv('data2.txt', header=None, names=['number'], dtype={'number': 'int64'}, blocksize=None)

# 
print(df.head())


# 应用函数到每一行，并计算总和
count = df['number'].map(calculate, meta=('number', 'int64')).sum().compute()

print(count)


       number
0  1334259689
1  1108738542
2  -331276288
3 -1742890076
4   571301911
102898


5.Измерить время выполнения для каждого случая.

In [11]:
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
import dask.dataframe as dd

# 简单的顺序算法
def method1():
    def calculate(num):
        factors = []
        i = 2
        while i * i <= num:
            if num % i == 0:
                factors.append(i)
                num = num // i
            else:
                i += 1
        if num > 1:
            factors.append(num)
        return len(factors)

    start_time = time.time()
    result = 0
    with open('data2.txt', 'r') as file:
        for line in file:
            number = int(line.strip())
            result += calculate(number)
    end_time = time.time()
    print("顺序读取时间 Простое последовательное чтение:{:.2f} s".format(end_time - start_time))




# 使用 Dask 的方法
def method3():
    def calculate(num):
        factors = []
        i = 2
        while i * i <= num:
            if num % i == 0:
                factors.append(i)
                num = num // i
            else:
                i += 1
        if num > 1:
            factors.append(num)
        return len(factors)

    start_time = time.time()
    df = dd.read_csv('data2.txt', header=None, names=['number'], dtype={'number': 'int64'}, blocksize=None)
    count = df['number'].map(calculate, meta=('number', 'int64')).sum().compute()
    
    end_time = time.time()
    
    print("使用Dask с помощью Ray/Dask/PySpark:{:.2f} s".format(end_time - start_time))


method1()
print('\n')
method3()


顺序读取时间 Простое последовательное чтение:7.50 s


使用Dask с помощью Ray/Dask/PySpark:7.25 s
